<a href="https://colab.research.google.com/github/jplee2/ds_module_22_big_data/blob/main/Home_Sales_starter_code_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,027 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy 

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
average_price_query = """
SELECT YEAR(date_sold) AS year, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date_sold)
ORDER BY year
"""

average_price_df = spark.sql(average_price_query)
average_price_df.show()


In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
average_price_beds_baths_query = """
SELECT YEAR(date_built) AS year, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY YEAR(date_built)
ORDER BY year
"""

average_price_beds_baths_df = spark.sql(average_price_beds_baths_query)
average_price_beds_baths_df.show()


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
average_price_query_floors_sqft = """
SELECT YEAR(date_built) AS year, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY YEAR(date_built)
ORDER BY year
"""

average_price_floors_sqft_df = spark.sql(average_price_query_floors_sqft)
average_price_floors_sqft_df.show()


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
import time

# Start the timer
start_time = time.time()

average_price_view_query = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

average_price_view_df = spark.sql(average_price_view_query)

# Show the results
average_price_view_df.show()

# Calculate the run time
end_time = time.time()
run_time = end_time - start_time
print(f"Query runtime: {run_time:.2f} seconds")

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# Start the timer for the cached query
start_time_cached = time.time()

# Run the query using the cached data
average_price_view_cached_df = spark.sql(average_price_view_query)

# Show the results
average_price_view_cached_df.show()

# Calculate the run time for the cached query
end_time_cached = time.time()
cached_run_time = end_time_cached - start_time_cached
print(f"Cached query runtime: {cached_run_time:.2f} seconds")

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Write the DataFrame to Parquet format, partitioned by the "date_built" field
df.write.partitionBy("date_built").parquet("home_sales_parquet", mode="overwrite")


In [ ]:
# 11. Read the parquet formatted data.
# Read the parquet formatted data into a DataFrame
parquet_df = spark.read.parquet("home_sales_parquet")

# Show the DataFrame to verify the data
parquet_df.show()

In [ ]:
# 12. Create a temporary view for the parquet DataFrame
parquet_df.createOrReplaceTempView("home_sales_parquet_view")


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# Start the timer for the parquet query
start_time_parquet = time.time()

# Run the query using the parquet DataFrame
average_price_view_parquet_query = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales_parquet_view
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

average_price_view_parquet_df = spark.sql(average_price_view_parquet_query)

# Show the results
average_price_view_parquet_df.show()

# Calculate the run time for the parquet query
end_time_parquet = time.time()
parquet_run_time = end_time_parquet - start_time_parquet
print(f"Parquet query runtime: {parquet_run_time:.2f} seconds")

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")